In [0]:
import pandas as pd
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
import numpy as np
from sklearn.metrics import mean_squared_error

In [0]:
df = pd.read_csv('cleaned_final.csv')

In [0]:
#Function to implement the SARIMA for hospitals

def myfunc(hospitalname):
  hosp = df[['Month', hospitalname]]
  hosp_sorted= hosp.sort_values('Month')
  hosp_sorted = hosp_sorted.set_index('Month')
 
  hosps= hosp_sorted

  p = d = q = range(0, 2)

  pdq = list(itertools.product(p, d, q))

  # Generating and printing all different combinations of p, q and q for trend and Seasonal
  seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]


  warnings.filterwarnings("ignore") # specify to ignore warning messages
  aicl=[]
  paraml=[]
  paramsl=[]

  for param in pdq:
      for param_seasonal in seasonal_pdq:
          try:
              mod = sm.tsa.statespace.SARIMAX(hosps,
                                              order=param,
                                              seasonal_order=param_seasonal,
                                              enforce_stationarity=False,
                                              enforce_invertibility=False)

              results = mod.fit()

              #print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
              aicl.append(results.aic)
              paraml.append(param)
              paramsl.append(param_seasonal)
              #mix[1] = aicl + paraml
          except:
              continue
  index=np.argmin(aicl)
  paraml[index][2]
  p1= paraml[index][0]
  d1=paraml[index][1]
  q1=paraml[index][2]
  p2= paramsl[index][0]
  d2= paramsl[index][1]
  q2= paramsl[index][2]
  mod_ARIMA = sm.tsa.statespace.SARIMAX(hosps,
                                  order=(p1, d1, q1),
                                  seasonal_order=(p2,d2,q2, 12),
                                  enforce_stationarity=False,
                                  enforce_invertibility=False)
  results = mod_ARIMA.fit()
  future_predictions = results.forecast(steps=32)
  future_predictions


  return future_predictions

In [0]:
#Manually computing the first hospital as we wanted the Date Column

function_values = myfunc('Ayr Hospital')
pred_df = pd.Series.to_frame(function_values)
df_1 = pd.DataFrame(pred_df).reset_index()
df_1.columns = ['Date','Ayr Hospital']


In [0]:
#Function to convert series into dataframe
#This function will then convert the series output into dataframe without the Date Column

def SeriesToFrame(df):
  function_values = myfunc(df)
  pred_df = pd.Series.to_frame(function_values)
  new = pd.DataFrame(pred_df).reset_index()
  new.columns = ['Date',df]
  new.drop('Date',axis=1,inplace=True)
  return new


In [0]:
# Calling all the hospitals

df_2 = SeriesToFrame('BARCALDINE HOSPITAL')
df_3 = SeriesToFrame('BLACKALL HOSPITAL')
df_4 = SeriesToFrame('Beaudesert Hospital')
df_5 = SeriesToFrame('Biloela Hospital')
df_6 = SeriesToFrame('Caloundra Hospital')
df_7 = SeriesToFrame('Capricorn Coast Hospital')
df_8 = SeriesToFrame('Charleville Hospital')
df_9 = SeriesToFrame('Charters Towers Hospital')
df_10 = SeriesToFrame('Cherbourg Hospital')
df_11 = SeriesToFrame('Chinchilla Hospital')
df_12 = SeriesToFrame('Cooktown Hospital')
df_13 = SeriesToFrame('Cunnamulla Hospital')
df_14 = SeriesToFrame('Dalby Hospital')
df_15 = SeriesToFrame('Emerald Hospital')
df_16 = SeriesToFrame('Goondiwindi Hospital')
df_17 = SeriesToFrame('Ingham Hospital')
df_18 = SeriesToFrame('Innisfail Hospital')
df_19 = SeriesToFrame('Joyce Palmer Health Service')
df_20 = SeriesToFrame('Kingaroy Hospital')
df_21 = SeriesToFrame('Longreach Hospital')
df_22 = SeriesToFrame('Oakey Hospital')
df_23 = SeriesToFrame('Proserpine Hospital')
df_24 = SeriesToFrame('Roma Hospital')
df_25 = SeriesToFrame('St George Hospital')
df_26 = SeriesToFrame('Stanthorpe Hospital')
df_27 = SeriesToFrame('Thursday Island Hospital')
df_28 = SeriesToFrame('WINTON  HOSPITAL')
df_29 = SeriesToFrame('Warwick Hospital')
df_30 = SeriesToFrame('Weipa Hospital')

In [0]:
#Collecting all the dataframes in one df
frames = [df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,df_21,df_22,df_23,df_24,df_25,df_26,df_27,df_28,df_29,df_30]

In [0]:
#Concatenationg all the dataframes
result = pd.concat(frames,axis=1)

In [0]:
#Replacing the negative values with the average of the column
result['BARCALDINE HOSPITAL'][result['BARCALDINE HOSPITAL'] < 0 ] = result['BARCALDINE HOSPITAL'].mean()


In [0]:
#Replacing the negative values with the average of the column
result['BLACKALL HOSPITAL'][result['BLACKALL HOSPITAL'] < 0 ] = result['BLACKALL HOSPITAL'].mean()


In [0]:
#Saving the results to an excel file
result.to_excel("Predictions.xlsx",sheet_name='Sheet_name_1',index=False) 